# imports/setup

In [73]:
import datetime
import json
import pandas as pd
from collections import defaultdict
from typing import List, Dict, Any
import string
import jsonpickle
import numpy as np
import math
import io
from datamodel import Listing, Trade, Observation
from datamodel import OrderDepth, UserId, TradingState, Order, ConversionObservation

# backtester

In [74]:
class Backtester:
    def __init__(self, trader, listings: Dict[str, Listing], position_limit: Dict[str, int],
                 market_data: pd.DataFrame, trade_history: pd.DataFrame, file_name: str = None):
        self.trader = trader
        self.listings = listings
        self.market_data = market_data
        self.position_limit = position_limit
        self.fair_marks = {}
        self.trade_history = trade_history.sort_values(by=['timestamp', 'symbol'])
        self.file_name = file_name

        self.observations = [Observation({}, {}) for _ in range(len(market_data))]

        self.current_position = {product: 0 for product in self.listings.keys()}
        self.pnl_history = []
        self.pnl = {product: 0 for product in self.listings.keys()}
        self.cash = {product: 0 for product in self.listings.keys()}
        self.trades = []
        self.sandbox_logs = []
        
    def run(self):
        traderData = ""
        
        timestamp_group_md = self.market_data.groupby('timestamp')
        timestamp_group_th = self.trade_history.groupby('timestamp')
        
        own_trades = defaultdict(list)
        market_trades = defaultdict(list)
        pnl_product = defaultdict(float)
        
        trade_history_dict = {}
        
        for timestamp, group in timestamp_group_th:
            trades = []
            for _, row in group.iterrows():
                symbol = row['symbol']
                price = row['price']
                quantity = row['quantity']
                buyer = row['buyer'] if pd.notnull(row['buyer']) else ""
                seller = row['seller'] if pd.notnull(row['seller']) else ""

                
                trade = Trade(symbol, int(price), int(quantity), buyer, seller, timestamp)
                
                trades.append(trade)
            trade_history_dict[timestamp] = trades
        
        
        for timestamp, group in timestamp_group_md:
            order_depths = self._construct_order_depths(group)
            order_depths_matching = self._construct_order_depths(group)
            order_depths_pnl = self._construct_order_depths(group)
            state = self._construct_trading_state(traderData, timestamp, self.listings, order_depths, 
                                 dict(own_trades), dict(market_trades), self.current_position, self.observations)
            orders, conversions, traderData = self.trader.run(state)
            products = group['product'].tolist()
            sandboxLog = ""
            trades_at_timestamp = trade_history_dict.get(timestamp, [])

            for product in products:
                new_trades = []
                for order in orders.get(product, []):
                    trades_done, sandboxLog = self._execute_order(timestamp, order, order_depths_matching, self.current_position, self.cash, trade_history_dict, sandboxLog)
                    new_trades.extend(trades_done)
                if len(new_trades) > 0:
                    own_trades[product] = new_trades
            self.sandbox_logs.append({"sandboxLog": sandboxLog, "lambdaLog": "", "timestamp": timestamp})

            trades_at_timestamp = trade_history_dict.get(timestamp, [])
            if trades_at_timestamp:
                for trade in trades_at_timestamp:
                    product = trade.symbol
                    market_trades[product].append(trade)
            else: 
                for product in products:
                    market_trades[product] = []

            
            for product in products:
                self._mark_pnl(self.cash, self.current_position, order_depths_pnl, self.pnl, product)
                self.pnl_history.append(self.pnl[product])
            self._add_trades(own_trades, market_trades)
        return self._log_trades(self.file_name)
    
    
    def _log_trades(self, filename: str = None):
        if filename is None:
            return 

        self.market_data['profit_and_loss'] = self.pnl_history

        output = ""
        output += "Sandbox logs:\n"
        for i in self.sandbox_logs:
            output += json.dumps(i, indent=2) + "\n"

        output += "\n\n\n\nActivities log:\n"
        market_data_csv = self.market_data.to_csv(index=False, sep=";")
        market_data_csv = market_data_csv.replace("\r\n", "\n")
        output += market_data_csv

        output += "\n\n\n\nTrade History:\n"
        output += json.dumps(self.trades, indent=2)

        with open(filename, 'w') as file:
            file.write(output)

            
    def _add_trades(self, own_trades: Dict[str, List[Trade]], market_trades: Dict[str, List[Trade]]):
        products = set(own_trades.keys()) | set(market_trades.keys())
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in own_trades.get(product, [])])
        for product in products:
            self.trades.extend([self._trade_to_dict(trade) for trade in market_trades.get(product, [])])

    def _trade_to_dict(self, trade: Trade) -> dict[str, Any]:
        return {
            "timestamp": trade.timestamp,
            "buyer": trade.buyer,
            "seller": trade.seller,
            "symbol": trade.symbol,
            "currency": "SEASHELLS",
            "price": trade.price,
            "quantity": trade.quantity,
        }
        
    def _construct_trading_state(self, traderData, timestamp, listings, order_depths, 
                                 own_trades, market_trades, position, observations):
        state = TradingState(traderData, timestamp, listings, order_depths, 
                             own_trades, market_trades, position, observations)
        return state
    
        
    def _construct_order_depths(self, group):
        order_depths = {}
        for idx, row in group.iterrows():
            product = row['product']
            order_depth = OrderDepth()
            for i in range(1, 4):
                if f'bid_price_{i}' in row and f'bid_volume_{i}' in row:
                    bid_price = row[f'bid_price_{i}']
                    bid_volume = row[f'bid_volume_{i}']
                    if not pd.isna(bid_price) and not pd.isna(bid_volume):
                        order_depth.buy_orders[int(bid_price)] = int(bid_volume)
                if f'ask_price_{i}' in row and f'ask_volume_{i}' in row:
                    ask_price = row[f'ask_price_{i}']
                    ask_volume = row[f'ask_volume_{i}']
                    if not pd.isna(ask_price) and not pd.isna(ask_volume):
                        order_depth.sell_orders[int(ask_price)] = -int(ask_volume)
            order_depths[product] = order_depth
        return order_depths
    
        
        
    def _execute_buy_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]

        for price, volume in list(order_depth.sell_orders.items()):
            if price > order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(trade_volume + position[order.symbol]) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "SUBMISSION", "", timestamp))
                position[order.symbol] += trade_volume
                self.cash[order.symbol] -= price * trade_volume
                order_depth.sell_orders[price] += trade_volume
                order.quantity -= trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"
            

            if order_depth.sell_orders[price] == 0:
                del order_depth.sell_orders[price]
        
        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price < order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "SUBMISSION", "", timestamp))
                    order.quantity -= trade_volume
                    position[order.symbol] += trade_volume
                    self.cash[order.symbol] -= order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp

        return trades, sandboxLog
        
        
        
    def _execute_sell_order(self, timestamp, order, order_depths, position, cash, trade_history_dict, sandboxLog):
        trades = []
        order_depth = order_depths[order.symbol]
        
        for price, volume in sorted(order_depth.buy_orders.items(), reverse=True):
            if price < order.price or order.quantity == 0:
                break

            trade_volume = min(abs(order.quantity), abs(volume))
            if abs(position[order.symbol] - trade_volume) <= int(self.position_limit[order.symbol]):
                trades.append(Trade(order.symbol, price, trade_volume, "", "SUBMISSION", timestamp))
                position[order.symbol] -= trade_volume
                self.cash[order.symbol] += price * abs(trade_volume)
                order_depth.buy_orders[price] -= abs(trade_volume)
                order.quantity += trade_volume
            else:
                sandboxLog += f"\nOrders for product {order.symbol} exceeded limit of {self.position_limit[order.symbol]} set"

            if order_depth.buy_orders[price] == 0:
                del order_depth.buy_orders[price]

        trades_at_timestamp = trade_history_dict.get(timestamp, [])
        new_trades_at_timestamp = []
        for trade in trades_at_timestamp:
            if trade.symbol == order.symbol:
                if trade.price > order.price:
                    trade_volume = min(abs(order.quantity), abs(trade.quantity))
                    trades.append(Trade(order.symbol, order.price, trade_volume, "", "SUBMISSION", timestamp))
                    order.quantity += trade_volume
                    position[order.symbol] -= trade_volume
                    self.cash[order.symbol] += order.price * trade_volume
                    if trade_volume == abs(trade.quantity):
                        continue
                    else:
                        new_quantity = trade.quantity - trade_volume
                        new_trades_at_timestamp.append(Trade(order.symbol, order.price, new_quantity, "", "", timestamp))
                        continue
            new_trades_at_timestamp.append(trade)  

        if len(new_trades_at_timestamp) > 0:
            trade_history_dict[timestamp] = new_trades_at_timestamp
                
        return trades, sandboxLog
        
        
        
    def _execute_order(self, timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog):
        if order.quantity == 0:
            return [], sandboxLog
        
        order_depth = order_depths[order.symbol]
        if order.quantity > 0:
            return self._execute_buy_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
        else:
            return self._execute_sell_order(timestamp, order, order_depths, position, cash, trades_at_timestamp, sandboxLog)
    
    def _mark_pnl(self, cash, position, order_depths, pnl, product):
        order_depth = order_depths[product]
        
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        mid = (best_ask + best_bid)/2
        fair = mid
        if product in self.fair_marks:
            get_fair = self.fair_marks[product]
            fair = get_fair(order_depth)
        
        pnl[product] = cash[product] + fair * position[product]

# trader

In [75]:
from datamodel import OrderDepth, UserId, TradingState, Order, ConversionObservation
from typing import List, Dict, Any, Deque, Optional

class Product:
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    PICNIC_BASKET1 = "PICNIC_BASKET1"
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    KELP = "KELP"
    RAINFOREST_RESIN = "RAINFOREST_RESIN"
    SQUID_INK = "SQUID_INK"
    SYNTHETIC = "SYNTHETIC"
    SPREAD = "SPREAD"


# Updated PARAMS to include separate parameters for each basket's spread
PARAMS = {
    Product.PICNIC_BASKET2: {
        "default_spread_mean": 36.46,
        "default_spread_std": 52.15,
        "spread_std_window": 30,
        "zscore_threshold": 3,
        "target_position": 88,
    },
}

# Basket compositions
BASKET_WEIGHTS = {
    Product.PICNIC_BASKET2: {
        Product.CROISSANTS: 4,
        Product.JAMS: 2,
    },
}


class Trader:
    def __init__(self, params=None, trade_synthetic=False):
        if params is None:
            params = PARAMS
        self.params = params
        self.trade_synthetic = trade_synthetic  # flag

        self.LIMIT = {
            Product.PICNIC_BASKET2: 100,
            Product.CROISSANTS: 250,
            Product.JAMS: 350,
            Product.DJEMBES: 60,
        }

    def get_swmid(self, order_depth) -> float:
        best_bid = max(order_depth.buy_orders.keys())
        best_ask = min(order_depth.sell_orders.keys())
        best_bid_vol = abs(order_depth.buy_orders[best_bid])
        best_ask_vol = abs(order_depth.sell_orders[best_ask])
        return (best_bid * best_ask_vol + best_ask * best_bid_vol) / (
            best_bid_vol + best_ask_vol
        )

    def get_synthetic_basket_order_depth(
        self, order_depths: Dict[str, OrderDepth], basket_type: str
    ) -> OrderDepth:
        # Get the basket composition
        basket_composition = BASKET_WEIGHTS[basket_type]

        # Initialize the synthetic basket order depth
        synthetic_order_price = OrderDepth()

        # Track the best bids and asks for each component in the basket
        component_best_bids = {}
        component_best_asks = {}

        # Calculate the best bid and ask for each component
        for component, weight in basket_composition.items():
            if component in order_depths and order_depths[component].buy_orders:
                component_best_bids[component] = max(
                    order_depths[component].buy_orders.keys()
                )
            else:
                component_best_bids[component] = 0

            if component in order_depths and order_depths[component].sell_orders:
                component_best_asks[component] = min(
                    order_depths[component].sell_orders.keys()
                )
            else:
                component_best_asks[component] = float("inf")

        # Calculate the implied bid and ask for the synthetic basket
        implied_bid = sum(
            component_best_bids[component] * weight
            for component, weight in basket_composition.items()
        )
        implied_ask = sum(
            component_best_asks[component] * weight
            for component, weight in basket_composition.items()
        )

        # Calculate the maximum number of synthetic baskets available at the implied bid and ask
        if implied_bid > 0:
            # Calculate how many baskets we can create based on each component's volume
            implied_bid_volumes = []
            for component, weight in basket_composition.items():
                if component_best_bids[component] > 0:
                    component_volume = order_depths[component].buy_orders[
                        component_best_bids[component]
                    ]
                    implied_bid_volumes.append(component_volume // weight)

            if implied_bid_volumes:  # Make sure we have volumes to calculate with
                implied_bid_volume = min(implied_bid_volumes)
                synthetic_order_price.buy_orders[implied_bid] = implied_bid_volume

        if implied_ask < float("inf"):
            # Calculate how many baskets we can create based on each component's volume
            implied_ask_volumes = []
            for component, weight in basket_composition.items():
                if component_best_asks[component] < float("inf"):
                    component_volume = -order_depths[component].sell_orders[
                        component_best_asks[component]
                    ]
                    implied_ask_volumes.append(component_volume // weight)

            if implied_ask_volumes:  # Make sure we have volumes to calculate with
                implied_ask_volume = min(implied_ask_volumes)
                synthetic_order_price.sell_orders[implied_ask] = -implied_ask_volume

        return synthetic_order_price

    def convert_synthetic_basket_orders(
        self,
        synthetic_orders: List[Order],
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
    ) -> Dict[str, List[Order]]:
        # Initialize the dictionary to store component orders
        component_orders = {component: [] for component in BASKET_WEIGHTS[basket_type]}

        # Get the best bid and ask for the synthetic basket
        synthetic_basket_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )
        best_bid = (
            max(synthetic_basket_order_depth.buy_orders.keys())
            if synthetic_basket_order_depth.buy_orders
            else 0
        )
        best_ask = (
            min(synthetic_basket_order_depth.sell_orders.keys())
            if synthetic_basket_order_depth.sell_orders
            else float("inf")
        )

        # Get the basket composition
        basket_composition = BASKET_WEIGHTS[basket_type]

        # Iterate through each synthetic basket order
        for order in synthetic_orders:
            # Extract the price and quantity from the synthetic basket order
            price = order.price
            quantity = order.quantity

            # Check if the synthetic basket order aligns with the best bid or ask
            if quantity > 0 and price >= best_ask:
                # Buy order - trade components at their best ask prices
                component_prices = {
                    component: min(order_depths[component].sell_orders.keys())
                    for component in basket_composition
                    if component in order_depths and order_depths[component].sell_orders
                }
            elif quantity < 0 and price <= best_bid:
                # Sell order - trade components at their best bid prices
                component_prices = {
                    component: max(order_depths[component].buy_orders.keys())
                    for component in basket_composition
                    if component in order_depths and order_depths[component].buy_orders
                }
            else:
                # The synthetic basket order does not align with the best bid or ask
                continue

            # Create orders for each component
            for component, weight in basket_composition.items():
                if component in component_prices:
                    component_order = Order(
                        component,
                        component_prices[component],
                        quantity * weight,
                    )
                    component_orders[component].append(component_order)

        return component_orders

    def execute_spread_orders(
        self,
        target_position: int,
        basket_position: int,
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
    ):
        if target_position == basket_position:
            return None

        target_quantity = abs(target_position - basket_position)
        basket_order_depth = order_depths[basket_type]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )

        if target_position > basket_position:
            if (
                not basket_order_depth.sell_orders
                or not synthetic_order_depth.buy_orders
            ):
                return None

            basket_ask_price = min(basket_order_depth.sell_orders.keys())
            basket_ask_volume = abs(basket_order_depth.sell_orders[basket_ask_price])

            synthetic_bid_price = max(synthetic_order_depth.buy_orders.keys())
            synthetic_bid_volume = abs(
                synthetic_order_depth.buy_orders[synthetic_bid_price]
            )

            orderbook_volume = min(basket_ask_volume, synthetic_bid_volume)
            execute_volume = min(orderbook_volume, target_quantity)

            basket_orders = [Order(basket_type, basket_ask_price, execute_volume)]
            if self.trade_synthetic:  # flag
                synthetic_orders = [
                    Order(Product.SYNTHETIC, synthetic_bid_price, -execute_volume)
                ]
                aggregate_orders = self.convert_synthetic_basket_orders(
                    synthetic_orders, order_depths, basket_type
                )
                aggregate_orders[basket_type] = basket_orders
            else:
                aggregate_orders = {basket_type: basket_orders}

            return aggregate_orders

        else:
            if (
                not basket_order_depth.buy_orders
                or not synthetic_order_depth.sell_orders
            ):
                return None

            basket_bid_price = max(basket_order_depth.buy_orders.keys())
            basket_bid_volume = abs(basket_order_depth.buy_orders[basket_bid_price])

            synthetic_ask_price = min(synthetic_order_depth.sell_orders.keys())
            synthetic_ask_volume = abs(
                synthetic_order_depth.sell_orders[synthetic_ask_price]
            )

            orderbook_volume = min(basket_bid_volume, synthetic_ask_volume)
            execute_volume = min(orderbook_volume, target_quantity)

            basket_orders = [Order(basket_type, basket_bid_price, -execute_volume)]
            if self.trade_synthetic:  # flag
                synthetic_orders = [
                    Order(Product.SYNTHETIC, synthetic_ask_price, execute_volume)
                ]
                aggregate_orders = self.convert_synthetic_basket_orders(
                    synthetic_orders, order_depths, basket_type
                )
                aggregate_orders[basket_type] = basket_orders
            else:
                aggregate_orders = {basket_type: basket_orders}

            return aggregate_orders

    def spread_orders(
        self,
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
        basket_position: int,
        spread_data: Dict[str, Any],
    ):
        if basket_type not in order_depths.keys():
            return None

        basket_order_depth = order_depths[basket_type]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )

        # Check if order depths have sufficient data
        if (
            not basket_order_depth.buy_orders
            or not basket_order_depth.sell_orders
            or not synthetic_order_depth.buy_orders
            or not synthetic_order_depth.sell_orders
        ):
            return None

        basket_swmid = self.get_swmid(basket_order_depth)
        synthetic_swmid = self.get_swmid(synthetic_order_depth)
        spread = basket_swmid - synthetic_swmid

        # Initialize the spread history if needed
        if f"{basket_type}_spread_history" not in spread_data:
            spread_data[f"{basket_type}_spread_history"] = []

        spread_data[f"{basket_type}_spread_history"].append(spread)

        spread_std_window = self.params[basket_type]["spread_std_window"]

        if len(spread_data[f"{basket_type}_spread_history"]) < spread_std_window:
            return None
        elif len(spread_data[f"{basket_type}_spread_history"]) > spread_std_window:
            spread_data[f"{basket_type}_spread_history"].pop(0)

        spread_std = np.std(spread_data[f"{basket_type}_spread_history"])

        # Use basket-specific parameters
        zscore = (spread - self.params[basket_type]["default_spread_mean"]) / spread_std

        if zscore >= self.params[basket_type]["zscore_threshold"]:
            if basket_position != -self.params[basket_type]["target_position"]:
                return self.execute_spread_orders(
                    -self.params[basket_type]["target_position"],
                    basket_position,
                    order_depths,
                    basket_type,
                )

        if zscore <= -self.params[basket_type]["zscore_threshold"]:
            if basket_position != self.params[basket_type]["target_position"]:
                return self.execute_spread_orders(
                    self.params[basket_type]["target_position"],
                    basket_position,
                    order_depths,
                    basket_type,
                )

        spread_data[f"{basket_type}_prev_zscore"] = zscore
        return None

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}
        conversions = 0

        # Initialize spread data if needed
        if Product.SPREAD not in traderObject:
            traderObject[Product.SPREAD] = {
                # Common spread data structure for all baskets
                # Individual basket data will be stored with basket-specific keys
                "clear_flag": False,
                "curr_avg": 0,
            }

        # Handle PICNIC_BASKET2 spread strategy
        if Product.PICNIC_BASKET2 in state.order_depths:
            basket_position = (
                state.position[Product.PICNIC_BASKET2]
                if Product.PICNIC_BASKET2 in state.position
                else 0
            )
            spread_orders = self.spread_orders(
                state.order_depths,
                Product.PICNIC_BASKET2,
                basket_position,
                traderObject[Product.SPREAD],
            )
            if spread_orders != None:
                if self.trade_synthetic:
                    for component in BASKET_WEIGHTS[Product.PICNIC_BASKET2]:
                        if component in spread_orders:
                            if component in result:
                                result[component].extend(
                                    spread_orders.get(component, [])
                                )
                            else:
                                result[component] = spread_orders.get(component, [])
                result[Product.PICNIC_BASKET2] = spread_orders.get(
                    Product.PICNIC_BASKET2, []
                )

        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData

# backtest run

In [76]:
def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

### setup

In [77]:
listings = {
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
}

position_limit = {
    'PICNIC_BASKET2': 60,
    'CROISSANTS': 250,
    'JAMS': 350,
    'DJEMBES': 60,
}

### RUN FROM DATABOTTLE

In [78]:
products_to_include = ["CROISSANTS", "JAMS", "DJEMBES", "PICNIC_BASKET2"]

day = 1
market_data = pd.read_csv(f"../../data/round4/prices_round_4_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"../../data/round4/trades_round_4_day_{day}_nn.csv", sep=";", header=0)

market_data = market_data[market_data["product"].isin(products_to_include)].copy()
trade_history = trade_history[trade_history["symbol"].isin(products_to_include)].copy()

trader = Trader()
backtester = Backtester(trader, listings, position_limit, market_data, trade_history, "trade_history_sim.log")
backtester.run()
print(backtester.pnl)

{'CROISSANTS': 0.0, 'JAMS': 0.0, 'DJEMBES': 0.0, 'PICNIC_BASKET2': 6863.0}


In [79]:
for i in range(1, 4):
    products_to_include = ["CROISSANTS", "JAMS", "DJEMBES", "PICNIC_BASKET2"]
    day = i
    market_data = pd.read_csv(f"../../data/round4/prices_round_4_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"../../data/round4/trades_round_4_day_{day}_nn.csv", sep=";", header=0)

    market_data = market_data[market_data["product"].isin(products_to_include)].copy()
    trade_history = trade_history[trade_history["symbol"].isin(products_to_include)].copy()

    trader = Trader()
    backtester = Backtester(trader, listings, position_limit, market_data, trade_history, f"clean_logs/trade_history_day_{i}.log")
    backtester.run()
    print(backtester.pnl)

{'CROISSANTS': 0.0, 'JAMS': 0.0, 'DJEMBES': 0.0, 'PICNIC_BASKET2': 6863.0}
{'CROISSANTS': 0.0, 'JAMS': 0.0, 'DJEMBES': 0.0, 'PICNIC_BASKET2': -7607.5}
{'CROISSANTS': 0.0, 'JAMS': 0.0, 'DJEMBES': 0.0, 'PICNIC_BASKET2': -8340.0}


# backtest gridsearch

In [80]:
def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [81]:
import os


def run_backtests(trader, listings, position_limit, market_data, trade_history, backtest_dir, param_grid, symbol):
    if not os.path.exists(backtest_dir):
        os.makedirs(backtest_dir)

    param_combinations = generate_param_combinations(param_grid[symbol])

    results = []
    for params in tqdm(param_combinations, desc=f"Running backtests for {symbol}", unit="backtest"):
        trader.params = {symbol: params}
        backtester = Backtester(trader, listings, position_limit, fair_calcs, market_data, trade_history)
        backtester.run()

        param_str = "-".join([f"{key}={value}" for key, value in params.items()])
        log_filename = f"{backtest_dir}/{symbol}_{param_str}.log"
        backtester._log_trades(log_filename)

        results.append((params, backtester.pnl[symbol]))

    return results

### setup

In [82]:
listings = {
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
}

position_limit = {
    'PICNIC_BASKET2': 100,
    'CROISSANTS': 250,
    'JAMS': 350,
    'DJEMBES': 60,
}

In [83]:
products_to_include = ["CROISSANTS", "JAMS", "DJEMBES", "PICNIC_BASKET2"]

day = 1
market_data = pd.read_csv(f"../../data/round4/prices_round_4_day_{day}.csv", sep=";", header=0)
trade_history = pd.read_csv(f"../../data/round4/trades_round_4_day_{day}_nn.csv", sep=";", header=0)

market_data = market_data[market_data["product"].isin(products_to_include)].copy()
trade_history = trade_history[trade_history["symbol"].isin(products_to_include)].copy()

### run

In [84]:
from datamodel import OrderDepth, TradingState, Order  # (Assumes these are provided elsewhere)
from typing import List, Dict, Any


class Product:
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    SYNTHETIC = "SYNTHETIC" # proxy basket
    SPREAD = "SPREAD" # basket - synthetic


PARAMS = {
    Product.PICNIC_BASKET2: {
        "default_spread_mean": 36.46,
        "default_spread_std": 52.15,
        "spread_std_window": 30,
        "zscore_threshold": 3,
        "target_position": 88,
    },
}

BASKET_WEIGHTS = {
    Product.PICNIC_BASKET2: {
        Product.CROISSANTS: 4,
        Product.JAMS: 2,
    },
}

class Trader:
    def __init__(self, params=None, trade_synthetic=False):
        if params is None:
            params = PARAMS
        self.params = params
        self.trade_synthetic = trade_synthetic  # flag

        self.LIMIT = {
            Product.PICNIC_BASKET2: 100,
            Product.CROISSANTS: 250,
            Product.JAMS: 350,
            Product.DJEMBES: 60,
        }

    def get_swmid(self, order_depth) -> float:
        best_bid = max(order_depth.buy_orders.keys())
        best_ask = min(order_depth.sell_orders.keys())
        best_bid_vol = abs(order_depth.buy_orders[best_bid])
        best_ask_vol = abs(order_depth.sell_orders[best_ask])
        return (best_bid * best_ask_vol + best_ask * best_bid_vol) / (
            best_bid_vol + best_ask_vol
        )

    def get_synthetic_basket_order_depth(
        self, order_depths: Dict[str, OrderDepth], basket_type: str
    ) -> OrderDepth:
        # Get the basket composition
        basket_composition = BASKET_WEIGHTS[basket_type]

        # Initialize the synthetic basket order depth
        synthetic_order_price = OrderDepth()

        # Track the best bids and asks for each component in the basket
        component_best_bids = {}
        component_best_asks = {}

        # Calculate the best bid and ask for each component
        for component, weight in basket_composition.items():
            if component in order_depths and order_depths[component].buy_orders:
                component_best_bids[component] = max(
                    order_depths[component].buy_orders.keys()
                )
            else:
                component_best_bids[component] = 0

            if component in order_depths and order_depths[component].sell_orders:
                component_best_asks[component] = min(
                    order_depths[component].sell_orders.keys()
                )
            else:
                component_best_asks[component] = float("inf")

        # Calculate the implied bid and ask for the synthetic basket
        implied_bid = sum(
            component_best_bids[component] * weight
            for component, weight in basket_composition.items()
        )
        implied_ask = sum(
            component_best_asks[component] * weight
            for component, weight in basket_composition.items()
        )

        # Calculate the maximum number of synthetic baskets available at the implied bid and ask
        if implied_bid > 0:
            # Calculate how many baskets we can create based on each component's volume
            implied_bid_volumes = []
            for component, weight in basket_composition.items():
                if component_best_bids[component] > 0:
                    component_volume = order_depths[component].buy_orders[
                        component_best_bids[component]
                    ]
                    implied_bid_volumes.append(component_volume // weight)

            if implied_bid_volumes:  # Make sure we have volumes to calculate with
                implied_bid_volume = min(implied_bid_volumes)
                synthetic_order_price.buy_orders[implied_bid] = implied_bid_volume

        if implied_ask < float("inf"):
            # Calculate how many baskets we can create based on each component's volume
            implied_ask_volumes = []
            for component, weight in basket_composition.items():
                if component_best_asks[component] < float("inf"):
                    component_volume = -order_depths[component].sell_orders[
                        component_best_asks[component]
                    ]
                    implied_ask_volumes.append(component_volume // weight)

            if implied_ask_volumes:  # Make sure we have volumes to calculate with
                implied_ask_volume = min(implied_ask_volumes)
                synthetic_order_price.sell_orders[implied_ask] = -implied_ask_volume

        return synthetic_order_price

    def convert_synthetic_basket_orders(
        self,
        synthetic_orders: List[Order],
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
    ) -> Dict[str, List[Order]]:
        # Initialize the dictionary to store component orders
        component_orders = {component: [] for component in BASKET_WEIGHTS[basket_type]}

        # Get the best bid and ask for the synthetic basket
        synthetic_basket_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )
        best_bid = (
            max(synthetic_basket_order_depth.buy_orders.keys())
            if synthetic_basket_order_depth.buy_orders
            else 0
        )
        best_ask = (
            min(synthetic_basket_order_depth.sell_orders.keys())
            if synthetic_basket_order_depth.sell_orders
            else float("inf")
        )

        # Get the basket composition
        basket_composition = BASKET_WEIGHTS[basket_type]

        # Iterate through each synthetic basket order
        for order in synthetic_orders:
            # Extract the price and quantity from the synthetic basket order
            price = order.price
            quantity = order.quantity

            # Check if the synthetic basket order aligns with the best bid or ask
            if quantity > 0 and price >= best_ask:
                # Buy order - trade components at their best ask prices
                component_prices = {
                    component: min(order_depths[component].sell_orders.keys())
                    for component in basket_composition
                    if component in order_depths and order_depths[component].sell_orders
                }
            elif quantity < 0 and price <= best_bid:
                # Sell order - trade components at their best bid prices
                component_prices = {
                    component: max(order_depths[component].buy_orders.keys())
                    for component in basket_composition
                    if component in order_depths and order_depths[component].buy_orders
                }
            else:
                # The synthetic basket order does not align with the best bid or ask
                continue

            # Create orders for each component
            for component, weight in basket_composition.items():
                if component in component_prices:
                    component_order = Order(
                        component,
                        component_prices[component],
                        quantity * weight,
                    )
                    component_orders[component].append(component_order)

        return component_orders

    def execute_spread_orders(
        self,
        target_position: int,
        basket_position: int,
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
    ):
        if target_position == basket_position:
            return None

        target_quantity = abs(target_position - basket_position)
        basket_order_depth = order_depths[basket_type]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )

        if target_position > basket_position:
            if (
                not basket_order_depth.sell_orders
                or not synthetic_order_depth.buy_orders
            ):
                return None

            basket_ask_price = min(basket_order_depth.sell_orders.keys())
            basket_ask_volume = abs(basket_order_depth.sell_orders[basket_ask_price])

            synthetic_bid_price = max(synthetic_order_depth.buy_orders.keys())
            synthetic_bid_volume = abs(
                synthetic_order_depth.buy_orders[synthetic_bid_price]
            )

            orderbook_volume = min(basket_ask_volume, synthetic_bid_volume)
            execute_volume = min(orderbook_volume, target_quantity)

            basket_orders = [Order(basket_type, basket_ask_price, execute_volume)]
            if self.trade_synthetic:  # flag
                synthetic_orders = [
                    Order(Product.SYNTHETIC, synthetic_bid_price, -execute_volume)
                ]
                aggregate_orders = self.convert_synthetic_basket_orders(
                    synthetic_orders, order_depths, basket_type
                )
                aggregate_orders[basket_type] = basket_orders
            else:
                aggregate_orders = {basket_type: basket_orders}

            return aggregate_orders

        else:
            if (
                not basket_order_depth.buy_orders
                or not synthetic_order_depth.sell_orders
            ):
                return None

            basket_bid_price = max(basket_order_depth.buy_orders.keys())
            basket_bid_volume = abs(basket_order_depth.buy_orders[basket_bid_price])

            synthetic_ask_price = min(synthetic_order_depth.sell_orders.keys())
            synthetic_ask_volume = abs(
                synthetic_order_depth.sell_orders[synthetic_ask_price]
            )

            orderbook_volume = min(basket_bid_volume, synthetic_ask_volume)
            execute_volume = min(orderbook_volume, target_quantity)

            basket_orders = [Order(basket_type, basket_bid_price, -execute_volume)]
            if self.trade_synthetic:  # flag
                synthetic_orders = [
                    Order(Product.SYNTHETIC, synthetic_ask_price, execute_volume)
                ]
                aggregate_orders = self.convert_synthetic_basket_orders(
                    synthetic_orders, order_depths, basket_type
                )
                aggregate_orders[basket_type] = basket_orders
            else:
                aggregate_orders = {basket_type: basket_orders}

            return aggregate_orders

    def spread_orders(
        self,
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
        basket_position: int,
        spread_data: Dict[str, Any],
    ):
        if basket_type not in order_depths.keys():
            return None

        basket_order_depth = order_depths[basket_type]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )

        # Check if order depths have sufficient data
        if (
            not basket_order_depth.buy_orders
            or not basket_order_depth.sell_orders
            or not synthetic_order_depth.buy_orders
            or not synthetic_order_depth.sell_orders
        ):
            return None

        basket_swmid = self.get_swmid(basket_order_depth)
        synthetic_swmid = self.get_swmid(synthetic_order_depth)
        spread = basket_swmid - synthetic_swmid

        # Initialize the spread history if needed
        if f"{basket_type}_spread_history" not in spread_data:
            spread_data[f"{basket_type}_spread_history"] = []

        spread_data[f"{basket_type}_spread_history"].append(spread)

        spread_std_window = self.params[basket_type]["spread_std_window"]

        if len(spread_data[f"{basket_type}_spread_history"]) < spread_std_window:
            return None
        elif len(spread_data[f"{basket_type}_spread_history"]) > spread_std_window:
            spread_data[f"{basket_type}_spread_history"].pop(0)

        spread_std = np.std(spread_data[f"{basket_type}_spread_history"])

        # Use basket-specific parameters
        zscore = (spread - self.params[basket_type]["default_spread_mean"]) / spread_std

        if zscore >= self.params[basket_type]["zscore_threshold"]:
            if basket_position != -self.params[basket_type]["target_position"]:
                return self.execute_spread_orders(
                    -self.params[basket_type]["target_position"],
                    basket_position,
                    order_depths,
                    basket_type,
                )

        if zscore <= -self.params[basket_type]["zscore_threshold"]:
            if basket_position != self.params[basket_type]["target_position"]:
                return self.execute_spread_orders(
                    self.params[basket_type]["target_position"],
                    basket_position,
                    order_depths,
                    basket_type,
                )

        spread_data[f"{basket_type}_prev_zscore"] = zscore
        return None

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}
        conversions = 0

        # Initialize spread data if needed
        if Product.SPREAD not in traderObject:
            traderObject[Product.SPREAD] = {
                # Common spread data structure for all baskets
                # Individual basket data will be stored with basket-specific keys
                "clear_flag": False,
                "curr_avg": 0,
            }

        # Handle PICNIC_BASKET2 spread strategy
        if Product.PICNIC_BASKET2 in state.order_depths:
            basket_position = (
                state.position[Product.PICNIC_BASKET2]
                if Product.PICNIC_BASKET2 in state.position
                else 0
            )
            spread_orders = self.spread_orders(
                state.order_depths,
                Product.PICNIC_BASKET2,
                basket_position,
                traderObject[Product.SPREAD],
            )
            if spread_orders != None:
                if self.trade_synthetic:
                    for component in BASKET_WEIGHTS[Product.PICNIC_BASKET2]:
                        if component in spread_orders:
                            if component in result:
                                result[component].extend(
                                    spread_orders.get(component, [])
                                )
                            else:
                                result[component] = spread_orders.get(component, [])
                result[Product.PICNIC_BASKET2] = spread_orders.get(
                    Product.PICNIC_BASKET2, []
                )

        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData

## analyze

In [85]:
import json
import pandas as pd
from collections import defaultdict
import jsonpickle
import numpy as np
import io
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation

In [86]:
class Product:
    PICNIC_BASKET2 = "PICNIC_BASKET2"
    CROISSANTS = "CROISSANTS"
    JAMS = "JAMS"
    DJEMBES = "DJEMBES"
    SYNTHETIC = "SYNTHETIC" # proxy basket
    SPREAD = "SPREAD" # basket - synthetic

def create_params(std_window, zscore_threshold, target_position):
    return {
        Product.PICNIC_BASKET2:{
            "default_spread_mean": 36.46,
            "default_spread_std": 52.15,
            "spread_std_window": std_window,
            "zscore_threshold": zscore_threshold,
            "target_position": target_position
        },
    }

In [89]:
from src.opti.backtester import Backtester
from tqdm import tqdm
import itertools

def _process_data_(file):
    with open(file, 'r') as file:
        log_content = file.read()
    sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
    sandbox_log =  sections[0].strip()
    activities_log = sections[1].split('Trade History:')[0]
    # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
    trade_history =  json.loads(sections[1].split('Trade History:')[1])
    # sandbox_log_df = pd.DataFrame(sandbox_log_list)
    market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
    trade_history_df = pd.json_normalize(trade_history)
    return market_data_df, trade_history_df

listings = {
    'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
    'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
    'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
    'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
}

position_limit = {
    'PICNIC_BASKET2': 100,
    'CROISSANTS': 250,
    'JAMS': 350,
    'DJEMBES': 60,
}

class Trader:
    def __init__(self, params=None, trade_synthetic=False):
        if params is None:
            params = PARAMS
        self.params = params
        self.trade_synthetic = trade_synthetic  # flag

        self.LIMIT = {
            Product.PICNIC_BASKET2: 100,
            Product.CROISSANTS: 250,
            Product.JAMS: 350,
            Product.DJEMBES: 60,
        }

    def get_swmid(self, order_depth) -> float:
        best_bid = max(order_depth.buy_orders.keys())
        best_ask = min(order_depth.sell_orders.keys())
        best_bid_vol = abs(order_depth.buy_orders[best_bid])
        best_ask_vol = abs(order_depth.sell_orders[best_ask])
        return (best_bid * best_ask_vol + best_ask * best_bid_vol) / (
            best_bid_vol + best_ask_vol
        )

    def get_synthetic_basket_order_depth(
        self, order_depths: Dict[str, OrderDepth], basket_type: str
    ) -> OrderDepth:
        # Get the basket composition
        basket_composition = BASKET_WEIGHTS[basket_type]

        # Initialize the synthetic basket order depth
        synthetic_order_price = OrderDepth()

        # Track the best bids and asks for each component in the basket
        component_best_bids = {}
        component_best_asks = {}

        # Calculate the best bid and ask for each component
        for component, weight in basket_composition.items():
            if component in order_depths and order_depths[component].buy_orders:
                component_best_bids[component] = max(
                    order_depths[component].buy_orders.keys()
                )
            else:
                component_best_bids[component] = 0

            if component in order_depths and order_depths[component].sell_orders:
                component_best_asks[component] = min(
                    order_depths[component].sell_orders.keys()
                )
            else:
                component_best_asks[component] = float("inf")

        # Calculate the implied bid and ask for the synthetic basket
        implied_bid = sum(
            component_best_bids[component] * weight
            for component, weight in basket_composition.items()
        )
        implied_ask = sum(
            component_best_asks[component] * weight
            for component, weight in basket_composition.items()
        )

        # Calculate the maximum number of synthetic baskets available at the implied bid and ask
        if implied_bid > 0:
            # Calculate how many baskets we can create based on each component's volume
            implied_bid_volumes = []
            for component, weight in basket_composition.items():
                if component_best_bids[component] > 0:
                    component_volume = order_depths[component].buy_orders[
                        component_best_bids[component]
                    ]
                    implied_bid_volumes.append(component_volume // weight)

            if implied_bid_volumes:  # Make sure we have volumes to calculate with
                implied_bid_volume = min(implied_bid_volumes)
                synthetic_order_price.buy_orders[implied_bid] = implied_bid_volume

        if implied_ask < float("inf"):
            # Calculate how many baskets we can create based on each component's volume
            implied_ask_volumes = []
            for component, weight in basket_composition.items():
                if component_best_asks[component] < float("inf"):
                    component_volume = -order_depths[component].sell_orders[
                        component_best_asks[component]
                    ]
                    implied_ask_volumes.append(component_volume // weight)

            if implied_ask_volumes:  # Make sure we have volumes to calculate with
                implied_ask_volume = min(implied_ask_volumes)
                synthetic_order_price.sell_orders[implied_ask] = -implied_ask_volume

        return synthetic_order_price

    def convert_synthetic_basket_orders(
        self,
        synthetic_orders: List[Order],
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
    ) -> Dict[str, List[Order]]:
        # Initialize the dictionary to store component orders
        component_orders = {component: [] for component in BASKET_WEIGHTS[basket_type]}

        # Get the best bid and ask for the synthetic basket
        synthetic_basket_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )
        best_bid = (
            max(synthetic_basket_order_depth.buy_orders.keys())
            if synthetic_basket_order_depth.buy_orders
            else 0
        )
        best_ask = (
            min(synthetic_basket_order_depth.sell_orders.keys())
            if synthetic_basket_order_depth.sell_orders
            else float("inf")
        )

        # Get the basket composition
        basket_composition = BASKET_WEIGHTS[basket_type]

        # Iterate through each synthetic basket order
        for order in synthetic_orders:
            # Extract the price and quantity from the synthetic basket order
            price = order.price
            quantity = order.quantity

            # Check if the synthetic basket order aligns with the best bid or ask
            if quantity > 0 and price >= best_ask:
                # Buy order - trade components at their best ask prices
                component_prices = {
                    component: min(order_depths[component].sell_orders.keys())
                    for component in basket_composition
                    if component in order_depths and order_depths[component].sell_orders
                }
            elif quantity < 0 and price <= best_bid:
                # Sell order - trade components at their best bid prices
                component_prices = {
                    component: max(order_depths[component].buy_orders.keys())
                    for component in basket_composition
                    if component in order_depths and order_depths[component].buy_orders
                }
            else:
                # The synthetic basket order does not align with the best bid or ask
                continue

            # Create orders for each component
            for component, weight in basket_composition.items():
                if component in component_prices:
                    component_order = Order(
                        component,
                        component_prices[component],
                        quantity * weight,
                    )
                    component_orders[component].append(component_order)

        return component_orders

    def execute_spread_orders(
        self,
        target_position: int,
        basket_position: int,
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
    ):
        if target_position == basket_position:
            return None

        target_quantity = abs(target_position - basket_position)
        basket_order_depth = order_depths[basket_type]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )

        if target_position > basket_position:
            if (
                not basket_order_depth.sell_orders
                or not synthetic_order_depth.buy_orders
            ):
                return None

            basket_ask_price = min(basket_order_depth.sell_orders.keys())
            basket_ask_volume = abs(basket_order_depth.sell_orders[basket_ask_price])

            synthetic_bid_price = max(synthetic_order_depth.buy_orders.keys())
            synthetic_bid_volume = abs(
                synthetic_order_depth.buy_orders[synthetic_bid_price]
            )

            orderbook_volume = min(basket_ask_volume, synthetic_bid_volume)
            execute_volume = min(orderbook_volume, target_quantity)

            basket_orders = [Order(basket_type, basket_ask_price, execute_volume)]
            if self.trade_synthetic:  # flag
                synthetic_orders = [
                    Order(Product.SYNTHETIC, synthetic_bid_price, -execute_volume)
                ]
                aggregate_orders = self.convert_synthetic_basket_orders(
                    synthetic_orders, order_depths, basket_type
                )
                aggregate_orders[basket_type] = basket_orders
            else:
                aggregate_orders = {basket_type: basket_orders}

            return aggregate_orders

        else:
            if (
                not basket_order_depth.buy_orders
                or not synthetic_order_depth.sell_orders
            ):
                return None

            basket_bid_price = max(basket_order_depth.buy_orders.keys())
            basket_bid_volume = abs(basket_order_depth.buy_orders[basket_bid_price])

            synthetic_ask_price = min(synthetic_order_depth.sell_orders.keys())
            synthetic_ask_volume = abs(
                synthetic_order_depth.sell_orders[synthetic_ask_price]
            )

            orderbook_volume = min(basket_bid_volume, synthetic_ask_volume)
            execute_volume = min(orderbook_volume, target_quantity)

            basket_orders = [Order(basket_type, basket_bid_price, -execute_volume)]
            if self.trade_synthetic:  # flag
                synthetic_orders = [
                    Order(Product.SYNTHETIC, synthetic_ask_price, execute_volume)
                ]
                aggregate_orders = self.convert_synthetic_basket_orders(
                    synthetic_orders, order_depths, basket_type
                )
                aggregate_orders[basket_type] = basket_orders
            else:
                aggregate_orders = {basket_type: basket_orders}

            return aggregate_orders

    def spread_orders(
        self,
        order_depths: Dict[str, OrderDepth],
        basket_type: str,
        basket_position: int,
        spread_data: Dict[str, Any],
    ):
        if basket_type not in order_depths.keys():
            return None

        basket_order_depth = order_depths[basket_type]
        synthetic_order_depth = self.get_synthetic_basket_order_depth(
            order_depths, basket_type
        )

        # Check if order depths have sufficient data
        if (
            not basket_order_depth.buy_orders
            or not basket_order_depth.sell_orders
            or not synthetic_order_depth.buy_orders
            or not synthetic_order_depth.sell_orders
        ):
            return None

        basket_swmid = self.get_swmid(basket_order_depth)
        synthetic_swmid = self.get_swmid(synthetic_order_depth)
        spread = basket_swmid - synthetic_swmid

        # Initialize the spread history if needed
        if f"{basket_type}_spread_history" not in spread_data:
            spread_data[f"{basket_type}_spread_history"] = []

        spread_data[f"{basket_type}_spread_history"].append(spread)

        spread_std_window = self.params[basket_type]["spread_std_window"]

        if len(spread_data[f"{basket_type}_spread_history"]) < spread_std_window:
            return None
        elif len(spread_data[f"{basket_type}_spread_history"]) > spread_std_window:
            spread_data[f"{basket_type}_spread_history"].pop(0)

        spread_std = np.std(spread_data[f"{basket_type}_spread_history"])

        # Use basket-specific parameters
        zscore = (spread - self.params[basket_type]["default_spread_mean"]) / spread_std

        if zscore >= self.params[basket_type]["zscore_threshold"]:
            if basket_position != -self.params[basket_type]["target_position"]:
                return self.execute_spread_orders(
                    -self.params[basket_type]["target_position"],
                    basket_position,
                    order_depths,
                    basket_type,
                )

        if zscore <= -self.params[basket_type]["zscore_threshold"]:
            if basket_position != self.params[basket_type]["target_position"]:
                return self.execute_spread_orders(
                    self.params[basket_type]["target_position"],
                    basket_position,
                    order_depths,
                    basket_type,
                )

        spread_data[f"{basket_type}_prev_zscore"] = zscore
        return None

    def run(self, state: TradingState):
        traderObject = {}
        if state.traderData != None and state.traderData != "":
            traderObject = jsonpickle.decode(state.traderData)

        result = {}
        conversions = 0

        # Initialize spread data if needed
        if Product.SPREAD not in traderObject:
            traderObject[Product.SPREAD] = {
                # Common spread data structure for all baskets
                # Individual basket data will be stored with basket-specific keys
                "clear_flag": False,
                "curr_avg": 0,
            }

        # Handle PICNIC_BASKET2 spread strategy
        if Product.PICNIC_BASKET2 in state.order_depths:
            basket_position = (
                state.position[Product.PICNIC_BASKET2]
                if Product.PICNIC_BASKET2 in state.position
                else 0
            )
            spread_orders = self.spread_orders(
                state.order_depths,
                Product.PICNIC_BASKET2,
                basket_position,
                traderObject[Product.SPREAD],
            )
            if spread_orders != None:
                if self.trade_synthetic:
                    for component in BASKET_WEIGHTS[Product.PICNIC_BASKET2]:
                        if component in spread_orders:
                            if component in result:
                                result[component].extend(
                                    spread_orders.get(component, [])
                                )
                            else:
                                result[component] = spread_orders.get(component, [])
                result[Product.PICNIC_BASKET2] = spread_orders.get(
                    Product.PICNIC_BASKET2, []
                )

        traderData = jsonpickle.encode(traderObject)

        return result, conversions, traderData

fair_calculations = {}

market_data0, trade_history0 = _process_data_('./clean_logs/trade_history_day_1.log')
market_data1, trade_history1 = _process_data_('./clean_logs/trade_history_day_2.log')
market_data2, trade_history2 = _process_data_('./clean_logs/trade_history_day_3.log')
market_data = market_data1
trade_history = trade_history1

std_windows = [30]
zscore_thresholds = [3]
target_positions = [10, 20, 30, 40, 50, 60, 70, 80]

results = []
for std_window, zscore_threshold, target_position in tqdm(itertools.product(std_windows, zscore_thresholds, target_positions)):
    total_pnl = 0
    params = create_params(std_window, zscore_threshold, target_position)
    trader = Trader(params=params)
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "simple_strat_backtest_no_clear_test_exceed.log")
    backtester.run()
    pnl = sum(float(pnl) for pnl in backtester.pnl.values())
    total_pnl += pnl

    results.append({
        "std_window": std_window,
        "zscore_threshold": zscore_threshold,
        "target_position": target_position,
        "pnl": total_pnl
    })
    print("="*80)
    print(f"std_window: {std_window}, zscore_threshold: {zscore_threshold}, target_position: {target_position}, pnl: {total_pnl}")
    print("="*80)

df_results = pd.DataFrame(results)
df_results.to_csv("optimization_results.csv", index=False)

1it [00:24, 24.16s/it]

std_window: 30, zscore_threshold: 3, target_position: 10, pnl: -1009.0


2it [00:45, 22.77s/it]

std_window: 30, zscore_threshold: 3, target_position: 20, pnl: -1781.0


3it [01:07, 22.11s/it]

...

std_window: 30, zscore_threshold: 3, target_position: 60, pnl: -4703.0


7it [02:31, 21.26s/it]

std_window: 30, zscore_threshold: 3, target_position: 70, pnl: -5552.0


8it [02:53, 21.68s/it]

std_window: 30, zscore_threshold: 3, target_position: 80, pnl: -6394.0
